In [ ]:
#From https://medium.com/@wwwanandsuresh/web-scraping-images-from-google-9084545808a2

In [1]:
!pip install requests

In [2]:
!pip install Pillow

In [1]:
import selenium
from selenium import webdriver
import time
import requests
import os
from PIL import Image
import io
import hashlib

In [2]:
#BELOW WORKS FOR CAKE - BUT RETURNS 16X16 ICONS AS WELL AS > THAN THE NUMBER ASKED FOR
import selenium
from selenium import webdriver
import time
import requests
import os
from PIL import Image
import io
import hashlib

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

# This is the path I use
#DRIVER_PATH = '/Users/anand/Desktop/chromedriver'
# Put the path for your ChromeDriver here
DRIVER_PATH = "chromedriver.exe"


def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        print(f'(Max links to fetch:{max_links_to_fetch})')
        scroll_to_end(wd)

        # get all image thumbnail results
        #thumbnail_results = wd.find_elements(By.CSS_SELECTOR, "Q4LuWd")
        #thumbnail_results = wd.find_elements(By.CSS_SELECTOR, "YQ4gaf")
        thumbnail_results = wd.find_elements(By.CLASS_NAME, "YQ4gaf")
        number_results = len(thumbnail_results)

        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")

        for img in thumbnail_results[results_start:number_results]:
        #for img in thumbnail_results[results_start:10]:

            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements(By.XPATH, "//img")
            print(f"Number of actual images is {len(actual_images)}")
            #print(f"The type of actual_images is {type(actual_images)}")


            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)
            print(f"The image count is: {image_count}")
            
            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

    # move the result startpoint further down
    results_start = len(thumbnail_results)

    return image_urls

def persist_image(folder_path:str,file_name:str,url:str):
    limit = 50
    final_images = 0
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    #while final_images <= limit:
    #while final_images <=50:
    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        width, height = image.size
        if width > 50:
            folder_path = os.path.join(folder_path,file_name)
            if os.path.exists(folder_path):
                file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
            else:
                os.mkdir(folder_path)
                file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
            #only save images that are > 10kb, or > 50x50 pixels               
            with open(file_path, 'wb') as f:
                image.save(f, "JPEG", quality=85)
            print(f"SUCCESS - saved {url} - as {file_path}")
            final_images += 1
#             if final_images == 50:
#                 break
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

if __name__ == '__main__':
    #wd = webdriver.Chrome(executable_path=DRIVER_PATH)
    service = Service(executable_path="chromedriver.exe")
    wd = webdriver.Chrome(service=service)
    #queries = ["CORONA-VIRUS","oneplus 8"]  #change your set of querries here
#    queries = ["bobotie", "borscht", "bouillabaisse", "brigadeiro", "brownies plain", "burrito"]
#    queries = ["egusi soup", "enchiladas"]
#    queries = ["falafel", "farofa", "fattoush", "feijoada", "fish and chips", "florentine cookies", "fried rice", "fufu", "gnocchi", "goulash"]
#    queries = ["bouillabaisse", "plain brownies", "chicken satay", "feijoada", "fufu"]
#    queries = ["minestrone"]
#    queries = ["mexican mole"]
#    queries = ["peking duck"]
#    queries = ["ramen"]
#    queries = ["miso soup", "mole", "moon cake", "mocqueca", "moroccan couscous", "moule frites", "okonomiyaki", "onigiri", "pad thai", "palak paneer"]
#    queries = ["peri peri chicken", "pho", "ramen"]
#    queries = ["stuffed cabbage", "sushi", "tacos", "tajine"]
    queries = ["samosa"]

    for query in queries:
        wd.get('https://google.com')
        search_box = wd.find_element(By.CLASS_NAME, 'gLFyf')
        search_box.send_keys(query)
        links = fetch_image_urls(query,2,wd)
        #images_path = '/Users/anand/Desktop/contri/images'  #enter your desired image path
        images_path = 'googleimages'
        if links:
            for i in links:
                persist_image(images_path,query,i)
        else:
            print("No links found")
    wd.quit()

(Max links to fetch:2)
Found: 452 search results. Extracting links from 0:452
Number of actual images is 507
The image count is: 320
Found: 320 image links, done!
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSQQu1u-sctfG8-qyOFxO8aMyeII2PSp-qv3Q&s - as googleimages\samosa\937f78004c.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSidcnA5RDnvoxr4nqhc50doflvzr_p1i1Xyg&s - as googleimages\samosa\c75f8c20af.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQioHXci-3aFdS9Pj8V7aMJ57uK24DG-jokFg&s - as googleimages\samosa\8508acc38a.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRa6R2ZImdwRgwHZzWIKPWjAkb_wTcj9PxnJA&s - as googleimages\samosa\ed93a1242e.jpg


C:\Users\keral\anaconda3\envs\capstone03\Lib\site-packages\PIL\Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQfJiSO8UR2Y-L9lYcIqe37UIiIVSQ10fnXUg&s - as googleimages\samosa\32e2d016d2.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSsq5WaioNFE03BNWiyXWugW18_l_HLsF-GWA&s - as googleimages\samosa\b86a510623.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRGTljGctSYsUwKusRSLHrr20WFnaHNcGk7MA&s - as googleimages\samosa\88cb99843f.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRtaGkCdiCsEYPE_QRjlmhGjCG4Q7Lkq4V6HQ&s - as googleimages\samosa\fcaba7e95c.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQkJCgrjSqDAC2ls-YTH2aIaoEuc52Tk8dgzw&s - as googleimages\samosa\74e44afbce.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTI2FQrOvNSB9nRQ_YNBE7F3kTasLyaz4-eSQ&s - as googleimages\samosa\09e31a8525.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTWt7ly9pZ1TNlhmXKllMgEJv-Z9cJ52Dr85g